In [1]:
import pandas as pd

df = pd.read_csv("snuc_carbon_year_2025.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df = df.sort_values(['Building_ID', 'Timestamp'])


In [2]:
SEQ_LEN = 168   # 7 days of hourly data


In [3]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(['Building_ID', 'Timestamp'])


In [4]:
def create_sequences(series, seq_len):
    X, y = [], []
    for i in range(len(series) - seq_len):
        X.append(series[i:i+seq_len])
        y.append(series[i+seq_len])
    return np.array(X), np.array(y)


In [6]:
pip install tensorflow[and-cuda]


Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement nvidia-nccl-cu12<3.0,>=2.25.1; extra == "and-cuda" (from tensorflow[and-cuda]) (from versions: 0.0.1.dev5)
ERROR: No matching distribution found for nvidia-nccl-cu12<3.0,>=2.25.1; extra == "and-cuda"


In [10]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.2-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached termcolor-3.2.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-2.0.1-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached grpcio-1.76.0-cp313-cp313-win_amd64.whl.metadata (3.8 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  U

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\mohit\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import numpy as np

SEQ_LEN = 168  # weekly window

lstm_models = {}
scalers = {}

for building in df['Building_ID'].unique():
    series = df[df['Building_ID'] == building]['Total_CO2e_kg'].values.reshape(-1,1)

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(series)

    X, y = create_sequences(scaled, SEQ_LEN)

    split = int(0.8 * len(X))
    X_train, y_train = X[:split], y[:split]
    X_val, y_val = X[split:], y[split:]

    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(SEQ_LEN,1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    lstm_models[building] = model
    scalers[building] = scaler


Epoch 1/30


C:\Users\mohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - loss: 0.0142 - val_loss: 9.9205e-04
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0080 - val_loss: 8.8438e-04
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 62ms/step - loss: 0.0070 - val_loss: 8.2206e-04
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0064 - val_loss: 7.8231e-04
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 76ms/step - loss: 0.0058 - val_loss: 8.5326e-04
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 66ms/step - loss: 0.0053 - val_loss: 7.6162e-04
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 65ms/step - loss: 0.0051 - val_loss: 7.8501e-04
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - loss: 0.0050 - val_loss: 7.7892e-04
Epoch 9/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0049 - val_loss: 7.6006e-04
Epoch 10/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0047 - val_loss: 7.6889e-04
Epoch 11/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - loss: 0.0046 - val_loss: 7.5145e-

C:\Users\mohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - loss: 0.0147 - val_loss: 8.7706e-04
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 65ms/step - loss: 0.0081 - val_loss: 0.0012
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0070 - val_loss: 0.0011
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - loss: 0.0062 - val_loss: 7.6386e-04
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step - loss: 0.0056 - val_loss: 0.0011
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 94ms/step - loss: 0.0055 - val_loss: 8.0527e-04
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 92ms/step - loss: 0.0053 - val_loss: 7.8581e-04
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - loss: 0.0052 - val_loss: 7.0839e-04
Epoch 9/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step - loss: 0.0049 - val_loss: 7.1783e-04
Epoch 10/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - loss: 0.0051 - val_loss: 7.8607e-04
Epoch 11/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 60ms/step - loss: 0.0047 - val_loss: 7.4821e-04
Epoch 12/

...

215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - loss: 0.0108 - val_loss: 0.0013
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 73ms/step - loss: 0.0067 - val_loss: 0.0011
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step - loss: 0.0060 - val_loss: 9.4815e-04
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 73ms/step - loss: 0.0056 - val_loss: 7.9537e-04
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 74ms/step - loss: 0.0055 - val_loss: 0.0010
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 3017s 14s/step - loss: 0.0054 - val_loss: 7.9748e-04
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - loss: 0.0054 - val_loss: 7.3067e-04
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 68ms/step - loss: 0.0053 - val_loss: 6.5802e-04
Epoch 9/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step - loss: 0.0053 - val_loss: 9.1285e-04
Epoch 10/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0052 - val_loss: 7.7739e-04
Epoch 11/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 62ms/step - loss: 0.0052 - val_loss: 6.4209e-04
Epoch 12

C:\Users\mohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - loss: 0.0119 - val_loss: 0.0017
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 61ms/step - loss: 0.0071 - val_loss: 0.0015
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - loss: 0.0063 - val_loss: 0.0018
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0058 - val_loss: 0.0010
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - loss: 0.0058 - val_loss: 8.7283e-04
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0055 - val_loss: 8.7246e-04
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0055 - val_loss: 7.6216e-04
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0053 - val_loss: 7.9405e-04
Epoch 9/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0053 - val_loss: 7.6927e-04
Epoch 10/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - loss: 0.0052 - val_loss: 8.2156e-04
Epoch 11/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - loss: 0.0051 - val_loss: 8.2888e-04
Epoch 12/30
2

C:\Users\mohit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 25s 102ms/step - loss: 0.0504 - val_loss: 0.0083
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0059 - val_loss: 0.0051
Epoch 3/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - loss: 0.0048 - val_loss: 0.0039
Epoch 4/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - loss: 0.0045 - val_loss: 0.0038
Epoch 5/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 99ms/step - loss: 0.0041 - val_loss: 0.0033
Epoch 6/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - loss: 0.0040 - val_loss: 0.0036
Epoch 7/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 88ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 8/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0037 - val_loss: 0.0030
Epoch 9/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0035 - val_loss: 0.0036
Epoch 10/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 95ms/step - loss: 0.0034 - val_loss: 0.0030
Epoch 11/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - loss: 0.0035 - val_loss: 0.0034
Epoch 12/30
215/215 ━━━━━━━━━━━━━━━━━━━━

Forecast 24 hrs

In [15]:
def forecast_next_24_lstm(building_id):
    model = lstm_models[building_id]
    scaler = scalers[building_id]

    series = df[df['Building_ID'] == building_id]['Total_CO2e_kg'].values.reshape(-1,1)
    scaled_series = scaler.transform(series)

    history = scaled_series[-SEQ_LEN:].reshape(1, SEQ_LEN, 1)
    forecast = {}

    last_time = df[df['Building_ID'] == building_id]['Timestamp'].iloc[-1]

    for i in range(24):
        pred = model.predict(history, verbose=0)[0][0]
        pred_real = scaler.inverse_transform([[pred]])[0][0]

        next_time = last_time + pd.Timedelta(hours=i+1)
        forecast[str(next_time)] = round(float(pred_real), 2)

        # roll window
        history = np.roll(history, -1, axis=1)
        history[0, -1, 0] = pred

    return forecast


JSON format

In [16]:
import json

forecast_output = {}

for building in df['Building_ID'].unique():
    forecast_output[building] = forecast_next_24_lstm(building)

with open("forecast_next_24_hours_lstm.json", "w") as f:
    json.dump(forecast_output, f, indent=4)

print("✅ LSTM forecast JSON generated")


✅ LSTM forecast JSON generated


Evaluation

In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [19]:
SEQ_LEN = 168   # same as training
HORIZON = 24    # evaluate next 24 hours


In [20]:
def evaluate_lstm(building_id, df, model, scaler):
    series = df[df['Building_ID'] == building_id]['Total_CO2e_kg'].values.reshape(-1,1)

    # Split: last 24 points = test
    train_series = series[:-HORIZON]
    test_series  = series[-HORIZON:]

    # Scale
    scaled_train = scaler.transform(train_series)
    scaled_test  = scaler.transform(test_series)

    # Initial history
    history = scaled_train[-SEQ_LEN:].reshape(1, SEQ_LEN, 1)
    predictions = []

    for i in range(HORIZON):
        pred = model.predict(history, verbose=0)[0][0]
        predictions.append(pred)

        # Teacher forcing: use ACTUAL value
        history = np.roll(history, -1, axis=1)
        history[0, -1, 0] = scaled_test[i][0]

    # Inverse transform
    y_pred = scaler.inverse_transform(np.array(predictions).reshape(-1,1)).flatten()
    y_true = test_series.flatten()

    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
    }


In [21]:
results = {}

for building in lstm_models:
    scores = evaluate_lstm(
        building,
        df,
        lstm_models[building],
        scalers[building]
    )
    results[building] = scores
    print(f"{building} → MAE: {scores['MAE']:.2f}, RMSE: {scores['RMSE']:.2f}")


Academic_Block_Large → MAE: 23.63, RMSE: 40.51
Academic_Block_Small → MAE: 6.94, RMSE: 9.41
Boys_Mess → MAE: 15.70, RMSE: 24.12
Canteen → MAE: 12.94, RMSE: 20.35
Clinic → MAE: 0.84, RMSE: 1.09
Girls_Mess → MAE: 19.62, RMSE: 30.75
Large_Hostel_Boys → MAE: 19.48, RMSE: 30.09
Large_Hostel_Girls → MAE: 24.72, RMSE: 35.42
Library → MAE: 17.09, RMSE: 27.51
Small_Hostel_Boys → MAE: 7.90, RMSE: 11.04
Small_Hostel_Girls → MAE: 8.77, RMSE: 11.84
Sports_Complex → MAE: 0.56, RMSE: 0.74
